In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import gridspec
import seaborn as sns

from scipy import signal
import random
import itertools

import pandas as pd
import xarray as xr
import numpy as np
import pickle

import warnings
from tqdm import tqdm
import os

from datetime import datetime

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.cross_decomposition import PLSRegression, CCA
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from MVA_algo_v2 import ReducedRankRegressor as RRR
from sklearn.metrics import r2_score

from data_treatment_tools import *
from anchor_models import *

In [ ]:
# ICML matplolib style
import matplotlib.pyplot as plt


from tueplots import bundles

bundles.icml2022()
{'text.usetex': True, 'font.family': 'serif', 'text.latex.preamble': '\\usepackage{times} ', 'figure.figsize': (3.25, 2.0086104634371584), 'figure.constrained_layout.use': True, 'figure.autolayout': False, 'savefig.bbox': 'tight', 'savefig.pad_inches': 0.015, 'font.size': 8, 'axes.labelsize': 8, 'legend.fontsize': 6, 'xtick.labelsize': 6, 'ytick.labelsize': 6, 'axes.titlesize': 8}

bundles.icml2022(family="sans-serif", usetex=False, column="full", nrows=2)
{'text.usetex': False, 'font.serif': ['Times'], 'mathtext.fontset': 'stix', 'mathtext.rm': 'Times', 'mathtext.it': 'Times:italic', 'mathtext.bf': 'Times:bold', 'font.family': 'sans-serif', 'figure.figsize': (6.75, 8.343458848123582), 'figure.constrained_layout.use': True, 'figure.autolayout': False, 'savefig.bbox': 'tight', 'savefig.pad_inches': 0.015, 'font.size': 8, 'axes.labelsize': 8, 'legend.fontsize': 6, 'xtick.labelsize': 6, 'ytick.labelsize': 6, 'axes.titlesize': 8}


# Plug any of those into either the rcParams or into an rc_context:

plt.rcParams.update(bundles.icml2022())

In [ ]:
import matplotlib.colors as mcolors
# Create a colormap object using 'coolwarm'
cmap = plt.cm.get_cmap('coolwarm')
blue_rgba1, blue_rgba2, red_rgba1, red_rgba2 = cmap(0.1), cmap(0.25), cmap(0.75), cmap(0.9)

In [ ]:
file_name = 'cmip56_data_coarse3_year.pkl' #File containing the model runs
directory = '../data/' 
# Selecting low DIV for train and high DIV for test
models_train_full = ['CCSM4', 'NorCPM1', 'CESM2', 'HadCM3']
models_test = ['CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']  #'IPSL-CM6A-LR', 
models = models_train_full + models_test
with open(directory+file_name, 'rb') as f:
    data = pickle.load(f)

# Loading useful variables
time = data['time']
lon = data['lon']
lat = data['lat']

### Computing DIV and GMT
* We extract the local cliamte response by averaging over all runs in each model*
* We obtain the Global response *gmt* by averaging the local responses spatially
* Decadal Internal Variability is computed by substracting the global response to the global temperature

In [ ]:
# Parameters of DIV, RMT and RIV
n_lat, n_lon, window_size = 30, 60, 10

# Adding ensemble average of tas (ea_tas) and global mean temperature (gmt)
for model in models :
    data[model]['ea_tas'] = data[model]['tas'].mean(axis=0)
    data[model]['gmt'] = data[model]['ea_tas'].mean(axis=(1, 2))

    
# Adding Decaal Internal Variability (div), Regional Mean Temperature (rmt) and Regional Internal variability (riv)
for model in models:
    div = decadalInternalVariability(data[model]['tas'], data[model]['gmt'])
    rmt, riv = regionalMeanTemperature(data[model]['tas'], n_lat, n_lon, window_size)
    data[model]['div'] = div
    data[model]['rmt'] = rmt
    data[model]['riv'] = riv

# Fingerprint

## Regional climate response

In [ ]:
years = 40
regional_trends = {}
for model in models:
    shape = data[model]['rmt'].shape
    Y = data[model]['rmt'][-years:,:,:].reshape(years, shape[1]*shape[2])
    X = np.arange(Y.shape[0])[:,None]
    trend = LinearRegression().fit(X, Y).coef_.reshape(shape[1], shape[2])
    regional_trends[model] = np.array(trend)

In [ ]:
# Plotting
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

contour = ax.pcolormesh(lon, lat, regional_trends[models[0]], transform=ccrs.PlateCarree(), cmap='coolwarm')

# Add coastlines and gridlines
ax.coastlines()
gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

plt.colorbar(contour, ax=ax, label='Error')

plt.title('Mean Error of ML Model on Temperature Predictions')
plt.show()


* We can see here that the temperature trends have a clear pattern. They are bigger in the poles (especially in the nothern pole) and lower when getting close to the equator.

In [ ]:
def extract_all_data(data, models, n_sample=1000, window_size=None):
    A = None
    for model in models:
        # Extracting data
        A_temp, X_temp, Y_temp = extract_data(data, model, window_size=None)
        selected_idx = random.sample(range(A_temp.shape[0]), n_sample)
        A_temp, X_temp, Y_temp = A_temp[selected_idx,:], X_temp[selected_idx,:], Y_temp[selected_idx,:]
        if A is None :
            A, X, Y = A_temp, X_temp, Y_temp
        else :
            A, X, Y = np.vstack((A, A_temp)), np.vstack((X, X_temp)), np.vstack((Y, Y_temp))
    return A, X, Y

def extract_data(data, model, n_lat_riv=None, n_lon_riv=None, window_size=None) :
    shape = data[model]['tas'].shape
    shape_rmt = data[model]['rmt'].shape
    
    Y = np.array([data[model]['rmt'] for i in range(shape[0])]).reshape(shape[0] * shape_rmt[0], shape_rmt[1]*shape_rmt[2])
    X = data[model]['tas'].reshape(shape[0] * shape[1], shape[2]*shape[3])
    A = data[model]['div'].reshape(shape[0] * shape[1], 1)

    if window_size is not None:
        # Create a 1D convolution kernel for the moving average
        kernel = np.ones(window_size) / window_size
        # Apply the moving average along axis=1
        Y = np.apply_along_axis(lambda x: np.convolve(x, kernel, mode='same'), axis=1, arr=Y)

    return A, X, Y


## Generating the train/test split
* We generate a set of training and validation models by randomly selecting two of the training models (used to train RRRR) and the two others are left for validation. 
* This procedure is run $B=10$ times

In [ ]:
B = 10
train_val_split = []
for b in range(B):
    idx_train = random.sample(range(len(models_train_full)), len(models_train_full)//2)
    idx_val = list(set(range(len(models_train_full))) - set(idx_train))
    train, val = [models_train_full[i] for i in idx_train], [models_train_full[i] for i in idx_val]
    train_val_split.append((train, val))

* We sequence the hyperpapramters

In [ ]:
#Selecting hyperparameters
sep_rank, min_rank, max_rank = 20, 200, 600
ranks = np.arange(min_rank, max_rank+1, sep_rank)
n_rank = len(ranks)

n_alpha = 20
alphas = np.logspace(0, 5, n_alpha)
n_sample = 500

* For each of the train/validation split we train an RRRR model for each combination of hyperparameters and validate it by computing the R2 score and the Mean correaltion between DIV and residuals

In [ ]:
# Stroring the MSE 
MSE_models = {'val' : [], 'train' : []}
for models_train, models_val in tqdm(train_val_split):
    # Extracting training/validation data
    A_train, X_train, Y_train = extract_all_data(data, models_train, n_sample=n_sample)
    A_val, X_val, Y_val = extract_all_data(data, models_val, n_sample=n_sample)
    # Normalizing data
    X_mean, Y_mean, A_mean = X_train.mean(axis=0), Y_train.mean(axis=0), A_train.mean(axis=0)
    X_std, Y_std, A_std = X_train.std(axis=0), Y_train.std(axis=0), A_train.std(axis=0)
    X_train_scaled, Y_train_scaled, A_train_scaled = (X_train - X_mean)/X_std, (Y_train - Y_mean)/Y_std, (A_train - A_mean)/A_std
    X_val_scaled, Y_val_scaled, A_val_scaled = (X_val - X_mean)/X_std, (Y_val - Y_mean)/Y_std, (A_val - A_mean)/A_std

    # Storing metrics 
    MSEs_val, MSE_PAs_val = np.zeros((n_rank, n_alpha)), np.zeros((n_rank, n_alpha))
    MSEs_train, MSE_PAs_train = np.zeros((n_rank, n_alpha)), np.zeros((n_rank, n_alpha))
    
    # Looping on hyperparameters (gamma and alpha)
    for i, rank in enumerate(ranks):
        for j, alpha in enumerate(alphas):
            # training a ridge model
            ridge = ReducedRankRegressor(rank=rank, reg=alpha)
            ridge.fit(X_train_scaled, Y_train_scaled)
            Y_pred_val = ridge.predict(X_val_scaled)
            Y_pred_train = ridge.predict(X_train_scaled)
            
            MSEs_val[i, j] = r2_score(Y_val_scaled, Y_pred_val)
            MSEs_train[i, j] = r2_score(Y_train_scaled, Y_pred_train)
            
    MSE_models['val'].append(MSEs_val)
    MSE_models['train'].append(MSEs_train)
            
# Averaging the scores
MSE_models['val'], MSE_models['train'] = np.array(MSE_models['val']), np.array(MSE_models['train'])

In [ ]:
MSE_models_mean = MSE_models['val'].mean(axis=0)
MSE_models_mean_train = MSE_models['train'].mean(axis=0)

* We select the optimal hyperparameters such that they minimize the $R^2$ score in validation

In [ ]:
xticklabels = ['{:.0e}'.format(i) for i in alphas[6:]]
yticklabels = ['{:.1e}'.format(i) for i in ranks]

plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1)
plt.title('MSE')
sns.heatmap(MSE_models_mean, xticklabels=xticklabels, yticklabels=yticklabels)
plt.xticks(fontsize=15, rotation =90)
plt.yticks(fontsize=15)

plt.show()

In [ ]:
optimal_idxs = np.unravel_index(MSE_models_mean.argmax(), MSE_models_mean.shape)
optimal_params = ranks[optimal_idxs[0]], alphas[optimal_idxs[1]]

print("Optimal hyperparameters (validation) :\n    * rank = {}\n    * alpha = {}".format(optimal_params[0], optimal_params[1]))

* We repeat the same procedure but now using the optimal hyperparameters selected above and use different values of $\gamma$ to assess the performance of anchor-regularized RRRR

In [ ]:
n_gamma = 20
gammas = np.logspace(0, 2, n_gamma)
gammas[7] = 5
# Stroring the MSE and MSE projected in the span of A (MSE_PA)
metrics = {'rrrr' : {'corr' : [], 'r2' : []}, 'anchor' : { gamma : {'corr' : [], 'r2' : []} for gamma in gammas}}
metrics_val = {'rrrr' : {'corr' : [], 'r2' : []}, 'anchor' : { gamma : {'corr' : [], 'r2' : []} for gamma in gammas}}

res_models = []
for models_train, models_val in tqdm(train_val_split):
    # Extracting training/validation data
    A_train, X_train, Y_train = extract_all_data(data, models_train, n_sample=n_sample)
    A_val, X_val, Y_val = extract_all_data(data, models_val, n_sample=n_sample)
    A_test, X_test, Y_test = extract_all_data(data, models_test, n_sample=n_sample)
    
    # Normalizing data
    X_mean, Y_mean, A_mean = X_train.mean(axis=0), Y_train.mean(axis=0), A_train.mean(axis=0)
    X_std, Y_std, A_std = X_train.std(axis=0), Y_train.std(axis=0), A_train.std(axis=0)
    X_train_scaled, Y_train_scaled, A_train_scaled = (X_train - X_mean)/X_std, (Y_train - Y_mean)/Y_std, (A_train - A_mean)/A_std
    X_val_scaled, Y_val_scaled, A_val_scaled = (X_val - X_mean)/X_std, (Y_val - Y_mean)/Y_std, (A_val - A_mean)/A_std
    X_test_scaled, Y_test_scaled, A_test_scaled = (X_test - X_mean)/X_std, (Y_test - Y_mean)/Y_std, (A_test - A_mean)/A_std
    
    # Creating Projector in span of A to project residual (P_A) - will be use to compute MSE_PA
    #P_A_val = A_val_scaled @ np.linalg.inv(A_val_scaled.T @ A_val_scaled) @ A_val_scaled.T
    #P_A_test = A_test_scaled @ np.linalg.inv(A_test_scaled.T @ A_test_scaled) @ A_test_scaled.T

    for gamma in gammas:
        # Training a RRRR
        anchor = ReducedRankRegressor(rank=optimal_params[0], reg=optimal_params[1])
        # Projecting the training data in anchor space
        AOP = AnchorOptimalProjection(gamma=gamma)
        X_train_transform, Y_train_transform = AOP.fit_transform(A_train_scaled, X_train_scaled, Y_train_scaled)

        # training a anchor model
        anchor.fit(X_train_transform, Y_train_transform)
        Y_pred_test_anchor = anchor.predict(X_test_scaled)
        metrics['anchor'][gamma]['r2'].append(r2_score(Y_test_scaled, Y_pred_test_anchor))
        metrics['anchor'][gamma]['corr'].append(np.array([np.corrcoef(a, res)[0, 1] for res in (Y_test_scaled - Y_pred_test_anchor).T for a in A_test_scaled.T]).mean())
        
        Y_pred_val_anchor = anchor.predict(X_val_scaled)
        metrics_val['anchor'][gamma]['r2'].append(r2_score(Y_val_scaled, Y_pred_val_anchor))
        metrics_val['anchor'][gamma]['corr'].append(np.array([np.corrcoef(a, res)[0, 1] for res in (Y_val_scaled - Y_pred_val_anchor).T for a in A_val_scaled.T]).mean())

In [ ]:
plt.figure(figsize=(10, 6))
values = np.array([metrics['anchor'][gamma]['r2'] for gamma in gammas]).T
plt.boxplot(values)
plt.xlabel(r'$\gamma$', fontsize=20)
plt.xticks(range(1, gammas.shape[0]+1), [round(gamma, 1) for gamma in gammas], rotation=45, fontsize=20)
plt.ylabel(r'$R^2$', fontsize=20)
plt.yticks(fontsize=20)
directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/R2_score_alpha.pdf", format="pdf", bbox_inches="tight")
plt.show()

* We can see that $\forall \gamma > 1$, anchor regularisation lead to better prediction performance on the testing set

In [ ]:
plt.figure(figsize=(10, 6))
values = np.array([metrics['anchor'][gamma]['corr'] for gamma in gammas]).T
plt.boxplot(values)
plt.xlabel(r'$\gamma$', fontsize=20)
plt.xticks(range(1, gammas.shape[0]+1), [round(gamma, 1) for gamma in gammas], rotation=45, fontsize=20)
plt.ylabel('Mean correlation between anchor and residuals', fontsize=16)
plt.yticks(fontsize=20)
directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/Correlation_score_alpha.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
r2_scores = np.mean([metrics['anchor'][gamma]['r2'] for gamma in gammas], axis=1)
mean_corr_scores = np.mean([metrics['anchor'][gamma]['corr'] for gamma in gammas], axis=1)
metric05 = 0.5*r2_scores/r2_scores.std() - 0.5*mean_corr_scores/mean_corr_scores.std()
metric09 = 0.9*r2_scores/r2_scores.std() - 0.1*mean_corr_scores/mean_corr_scores.std()
argmin05, argmin09 = np.argmax(metric05), np.argmax(metric09)

In [ ]:
r2_scores_val = np.mean([metrics_val['anchor'][gamma]['r2'] for gamma in gammas], axis=1)
mean_corr_scores_val = np.mean([metrics_val['anchor'][gamma]['corr'] for gamma in gammas], axis=1)
metric05_val = 0.5*r2_scores_val/r2_scores_val.std() - 0.5*mean_corr_scores_val/mean_corr_scores_val.std()
metric09_val = 0.9*r2_scores_val/r2_scores_val.std() - 0.1*mean_corr_scores_val/mean_corr_scores_val.std()
argmin05_val, argmin09_val = np.argmax(metric05_val), np.argmax(metric09_val)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Scatter plots
plt.scatter(r2_scores[argmin05], mean_corr_scores[argmin05], c=red_rgba1, s=300, marker="D", edgecolors='black', label=r'optimal $\gamma$ $0.5/0.5$ (test)')
plt.scatter(r2_scores[argmin09], mean_corr_scores[argmin09], c=red_rgba2, s=300, marker="D", edgecolors='black', label=r'optimal $\gamma$ $0.1/0.9$ (test)')
plt.scatter(r2_scores[0], mean_corr_scores[0], c=red_rgba1, s=300, marker="^", edgecolors='black', label=r'RRRR (test)')


plt.scatter(r2_scores_val[argmin05_val], mean_corr_scores_val[argmin05_val], c=blue_rgba2, s=300, marker="D", edgecolors='black', label=r'optimal $\gamma$ $0.5/0.5$ (train)')
plt.scatter(r2_scores_val[argmin09_val], mean_corr_scores_val[argmin09_val], c=blue_rgba1, s=300, marker="D", edgecolors='black', label=r'optimal $\gamma$ $0.1/0.9$ (train)')
plt.scatter(r2_scores_val[0], mean_corr_scores_val[0], c=blue_rgba1, s=300, marker="^", edgecolors='black', label=r'RRRR (train)')

# Line plot
plt.plot(r2_scores, mean_corr_scores, c=red_rgba1, marker='o', markersize=8, linewidth=4)
plt.plot(r2_scores_val, mean_corr_scores_val, c=blue_rgba2, marker='o', markersize=8, linewidth=4)

plt.ylabel('Mean correlation between DIV and residuals', fontsize=20)
plt.xlabel(r'$R^2$', fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)

# Legend above the plot
plt.legend(fontsize=20, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)

directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/pareto_gamma_opt_30x60.pdf", format="pdf", bbox_inches="tight")
plt.show()


* Anchor regularised RRRR show also a lower correlation between DIV and residuals 

# Scores maps
We know train A-RRRR on the full set of training models with optimal $(\alpha, r_r)$ parameters, now with only 250 samples for each model in order ot get the same number of samples as in validation.

In [ ]:
gamma = 5
res_map_anchor, corr_map_anchor = [], []
res_map_rrrr, corr_map_rrrr = [], []

B=10
for b in tqdm(range(B)):
    A_train, X_train, Y_train = extract_all_data(data, models_train_full, n_sample=n_sample//2)
    #A_val, X_val, Y_val = extract_all_data(data, models_val, n_sample=n_sample)
    A_test, X_test, Y_test = extract_all_data(data, models_test, n_sample=n_sample)

    # Normalizing data
    X_mean, Y_mean, A_mean = X_train.mean(axis=0), Y_train.mean(axis=0), A_train.mean(axis=0)
    X_std, Y_std, A_std = X_train.std(axis=0), Y_train.std(axis=0), A_train.std(axis=0)
    X_train_scaled, Y_train_scaled, A_train_scaled = (X_train - X_mean)/X_std, (Y_train - Y_mean)/Y_std, (A_train - A_mean)/A_std
    #X_val_scaled, Y_val_scaled, A_val_scaled = (X_val - X_mean)/X_std, (Y_val - Y_mean)/Y_std, (A_val - A_mean)/A_std
    X_test_scaled, Y_test_scaled, A_test_scaled = (X_test - X_mean)/X_std, (Y_test - Y_mean)/Y_std, (A_test - A_mean)/A_std
    anchor = ReducedRankRegressor(rank=optimal_params[0], reg=optimal_params[1])
    # Projecting the training data in anchor space
    AOP = AnchorOptimalProjection(gamma=gamma)
    X_train_transform, Y_train_transform = AOP.fit_transform(A_train_scaled, X_train_scaled, Y_train_scaled)

    # training a anchor model
    anchor.fit(X_train_transform, Y_train_transform)
    Y_pred_test_anchor = anchor.predict(X_test_scaled)
    res_anchor = Y_test_scaled - Y_pred_test_anchor
    res_anchor_map = r2_score(Y_test_scaled, Y_pred_test_anchor, multioutput='raw_values').reshape(n_lat, n_lon)
    corr_anchor = np.array([np.corrcoef(A_test_scaled[:,0], res)[0, 1] for res in res_anchor.T])
    corr_anchor_map = corr_anchor.reshape(n_lat, n_lon)
    
    res_map_anchor.append(res_anchor_map)
    corr_map_anchor.append(corr_anchor_map)
    
    # training a rrrr model
    rrrr = ReducedRankRegressor(rank=optimal_params[0], reg=optimal_params[1])
    rrrr.fit(X_train_scaled, Y_train_scaled)
    Y_pred_test_rrrr = rrrr.predict(X_test_scaled)
    res_rrrr = Y_test_scaled - Y_pred_test_rrrr
    res_rrrr_map = r2_score(Y_test_scaled, Y_pred_test_rrrr, multioutput='raw_values').reshape(n_lat, n_lon)
    corr_rrrr = np.array([np.corrcoef(A_test_scaled[:,0], res)[0, 1] for res in res_rrrr.T])
    corr_rrrr_map = corr_rrrr.reshape(n_lat, n_lon)
    
    res_map_rrrr.append(res_rrrr_map)
    corr_map_rrrr.append(corr_rrrr_map)

res_map_anchor, corr_map_anchor = np.array(res_map_anchor), np.array(corr_map_anchor)
res_map_rrrr, corr_map_rrrr = np.array(res_map_rrrr), np.array(corr_map_rrrr)

In [ ]:
res_map_anchor_mean, corr_map_anchor_mean = res_map_anchor.mean(axis=0), corr_map_anchor.mean(axis=0)
res_map_rrrr_mean, corr_map_rrrr_mean = res_map_rrrr.mean(axis=0), corr_map_rrrr.mean(axis=0)

In [ ]:
diff_r2_map = res_map_anchor_mean - res_map_rrrr_mean
diff_corr_map = abs(corr_map_rrrr_mean) - abs(corr_map_anchor_mean)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# Assuming you have defined lat, lon, diff_r2_map, diff_corr_map, max_val_r2, max_val_corr

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(20, 6), subplot_kw={'projection': ccrs.Robinson()})

# Plot for diff_r2_map
norm_r2 = plt.Normalize(vmin=-0.15, vmax=0.15)
contour_r2 = axs[0].pcolormesh(lon, lat, diff_r2_map, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_r2)
contour_r2_hatched = axs[0].contourf(lon, lat, diff_r2_map, levels=[0, max_val_r2], hatches=['.', None], colors='none', transform=ccrs.PlateCarree())
axs[0].coastlines()
gl_r2 = axs[0].gridlines(draw_labels=True)
gl_r2.top_labels = gl_r2.right_labels = False
gl_r2.xformatter = LONGITUDE_FORMATTER
gl_r2.yformatter = LATITUDE_FORMATTER
gl_r2.xlabel_style = {'size': 20}  # Longitude font size
gl_r2.ylabel_style = {'size': 20}  # Latitude font size
axs[0].set_title('A', fontsize=25)

cb2 = fig.colorbar(contour_r2, ax=axs[0], label='R2 score differences', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label(r'$\Delta R^2$ ', fontsize=20) 

# Plot for diff_corr_map
norm_corr = plt.Normalize(vmin=-0.4, vmax=0.4)
contour_corr = axs[1].pcolormesh(lon, lat, diff_corr_map, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_corr)
contour_corr_hatched = axs[1].contourf(lon, lat, diff_corr_map, levels=[0, max_val_corr], hatches=['.', None], colors='none', transform=ccrs.PlateCarree())
axs[1].coastlines()
gl_corr = axs[1].gridlines(draw_labels=True)
gl_corr.top_labels = gl_corr.right_labels = False
gl_corr.xformatter = LONGITUDE_FORMATTER
gl_corr.yformatter = LATITUDE_FORMATTER

gl_corr.xlabel_style = {'size': 20}  # Longitude font size
gl_corr.ylabel_style = {'size': 20}  # Latitude font size
axs[1].set_title('B', fontsize=25)

cb2 = fig.colorbar(contour_corr, ax=axs[1], label='Residuals', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label(r'$\Delta r$', fontsize=20)  # Colorbar label font size

directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/maps_R2_corr_30x60_gamma{}.pdf".format(gamma), format="pdf", bbox_inches="tight")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# Assuming you have defined lat, lon, diff_r2_map, diff_corr_map, max_val_r2, max_val_corr

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(20, 6), subplot_kw={'projection': ccrs.Robinson()})

# Plot for diff_r2_map
norm_r2 = TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.8)
contour_r2 = axs[0].pcolormesh(lon, lat, res_map_anchor_mean, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_r2)
axs[0].coastlines()
gl_r2 = axs[0].gridlines(draw_labels=True)
gl_r2.top_labels = gl_r2.right_labels = False
gl_r2.xformatter = LONGITUDE_FORMATTER
gl_r2.yformatter = LATITUDE_FORMATTER
gl_r2.xlabel_style = {'size': 20}  # Longitude font size
gl_r2.ylabel_style = {'size': 20}  # Latitude font size
axs[0].set_title('A', fontsize=25)

cb2 = fig.colorbar(contour_r2, ax=axs[0], label='R2 score differences', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label(r'$R^2$', fontsize=20) 

# Plot for diff_corr_map
norm_corr = plt.Normalize(vmin=-0.6, vmax=0.6)
contour_corr = axs[1].pcolormesh(lon, lat, corr_map_anchor_mean, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_corr)
axs[1].coastlines()
gl_corr = axs[1].gridlines(draw_labels=True)
gl_corr.top_labels = gl_corr.right_labels = False
gl_corr.xformatter = LONGITUDE_FORMATTER
gl_corr.yformatter = LATITUDE_FORMATTER

gl_corr.xlabel_style = {'size': 20}  # Longitude font size
gl_corr.ylabel_style = {'size': 20}  # Latitude font size
axs[1].set_title('B', fontsize=25)

cb2 = fig.colorbar(contour_corr, ax=axs[1], label='Residuals', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label('Correlation between DIV and residuals', fontsize=20)  # Colorbar label font size

directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/maps_anchor_R2_corr_30x60_gamma{}.pdf".format(gamma), format="pdf", bbox_inches="tight")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.colors import TwoSlopeNorm

# Assuming you have defined lat, lon, diff_r2_map, diff_corr_map, max_val_r2, max_val_corr

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(20, 6), subplot_kw={'projection': ccrs.Robinson()})

# Plot for diff_r2_map
norm_r2 = TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.8)
contour_r2 = axs[0].pcolormesh(lon, lat, res_map_rrrr_mean, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_r2)
axs[0].coastlines()
gl_r2 = axs[0].gridlines(draw_labels=True)
gl_r2.top_labels = gl_r2.right_labels = False
gl_r2.xformatter = LONGITUDE_FORMATTER
gl_r2.yformatter = LATITUDE_FORMATTER
gl_r2.xlabel_style = {'size': 20}  # Longitude font size
gl_r2.ylabel_style = {'size': 20}  # Latitude font size
axs[0].set_title('A', fontsize=25)

cb2 = fig.colorbar(contour_r2, ax=axs[0], label='R2 score differences', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label(r'$R^2$', fontsize=20) 

# Plot for diff_corr_map
norm_corr = plt.Normalize(vmin=-0.6, vmax=0.6)
contour_corr = axs[1].pcolormesh(lon, lat, corr_map_rrrr_mean, transform=ccrs.PlateCarree(), cmap='coolwarm', norm=norm_corr)
axs[1].coastlines()
gl_corr = axs[1].gridlines(draw_labels=True)
gl_corr.top_labels = gl_corr.right_labels = False
gl_corr.xformatter = LONGITUDE_FORMATTER
gl_corr.yformatter = LATITUDE_FORMATTER

gl_corr.xlabel_style = {'size': 20}  # Longitude font size
gl_corr.ylabel_style = {'size': 20}  # Latitude font size
axs[1].set_title('B', fontsize=25)

cb2 = fig.colorbar(contour_corr, ax=axs[1], label='Residuals', orientation='horizontal')
cb2.ax.tick_params(labelsize=20)
cb2.set_label('Correlation between DIV and residuals', fontsize=20)  # Colorbar label font size

directory = '/home/homer/Documents/Doctorado/Investigacion/AnchorMultivariateAnalysis/Results'
plt.savefig(directory + "/maps_rrrr_R2_corr_30x60.pdf", format="pdf", bbox_inches="tight")

plt.show()
